In [ ]:
#These were combined from multiple scripts to unify the naming scheme.

#For showing FTH1 results
import NanobodyPaperPlotting
import matplotlib.pyplot as plt
import fcsparser
import numpy as np
import seaborn as sns

colors_decimal = [[120/255, 120/255, 120/255], [249/255, 29/255, 0/255], [32/255, 25/255, 250/255]]
AmFRETLims_FTH1 = (-0.4, 1.0) #changed for this panel
expressionLims_FTH1 = (10**0.075, 10**3) #changed for this panel

#For showing nanobody tests
import NanobodyPaperPlotting
import matplotlib.pyplot as plt
import fcsparser
import numpy as np
from matplotlib.collections import LineCollection

colors_decimal = [[120/255, 120/255, 120/255], [249/255, 29/255, 0/255], [32/255, 25/255, 250/255]]

expressionLims_NB = (10**0.03, 10**3) #changed for this panel
AmFRETLims_NB = (-0.3, 1) #changed for this panel

#For showing BDFP gating
import NanobodyPaperPlotting
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

AmFRETLims = (-0.2, 1)

expressionLims_kde = (0.25, 3)
expressionLims = (10**0.25, 10**3)
BDFPPositiveBDFPSSCLims_kde = (-1, 3)
BDFPNegativeBDFPSSCLims_kde = (-4, 1)

defaultXLabel = "mEos3 concentration (p.d.u.)"
defaultYLabel = "BDFP1.6:1.6 concentration (p.d.u.)"

#for pLDDT and PAE colorbars
import matplotlib.pyplot as plt
import matplotlib.colors as colors

#the model and pae plot in supplemental figure 1 were made in chimerax, which uses set (and not entirely standard) color schemes, but we wanted colorbars.

### Figure S1

In [ ]:
fth1Files = [("FTH1 on", ["data/FTH1/A1.fcs"]), 
             ("FTH1 off", ["data/FTH1/D1.fcs"])]

fth1Data = []
#I'm copying some stuff from NanobodyPaperPlotting.readDataFromFilelist here to avoid making weird BDFP messes elsewhere.
for title, files in fth1Files:
    _, data = fcsparser.parse(files[0]) # for fth1, there is only one file

    #make both because both are useful
    data["Acceptor/SSC"] = data["Acceptor-A"] / data["SSC 488/10-A"]
    data["log(Acceptor/SSC)"] = np.log10(data["Acceptor/SSC"])

    fth1Data.append((title, data))

squarePlotSize = 3
numCols = 2
numRows = 1

DAmFRETFig = plt.Figure((squarePlotSize*numCols ,squarePlotSize*numRows), dpi=300)
DAmFRETAxs = DAmFRETFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
NanobodyPaperPlotting.DAmFRETRowFromData(fth1Data, DAmFRETAxs, addTitles=True, ylims=AmFRETLims_FTH1, xlims=expressionLims_FTH1, firstColLabelsOnly=True)

DAmFRETFig.tight_layout()
plt.show()
# DAmFRETFig.savefig("plots/FTH1Supplement_DAmFRET.png")  #NOTE: titles were changed in illustrator
#NOTE: this version of NanobodyPaperPlotting adds a thinner line than the version in the paper.
DAmFRETFig

In [ ]:
NBTestFiles = [("Control", ["data/NBTests/G3.fcs"]),
               ("mEosNb", ["data/NBTests/A12.fcs"]),
               ("mEosNb0", ["data/NBTests/D7.fcs"]),
               ("NbALFA", ["data/NBTests/B5.fcs"])]

NBTestData = []

#the control doesn't have BDFP, so it needs to be handled specially
_, data = fcsparser.parse(NBTestFiles[0][1][0])
#make both because both are useful
data["Acceptor/SSC"] = data["Acceptor-A"] / data["SSC 488/10-A"]
data["log(Acceptor/SSC)"] = np.log10(data["Acceptor/SSC"])

NBTestData.append((NBTestFiles[0][0], data))

for title, files in NBTestFiles[1:]:
    data = NanobodyPaperPlotting.readDataFromFilelist(files)
    NBTestData.append((title, data))

squarePlotSize = 3
numCols = 4
numRows = 1

DAmFRETFig = plt.Figure((squarePlotSize*numCols ,squarePlotSize*numRows), dpi=300)
DAmFRETAxs = DAmFRETFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
NanobodyPaperPlotting.DAmFRETRowFromData(NBTestData, DAmFRETAxs, addTitles=True, ylims=AmFRETLims_NB, firstColLabelsOnly=True, xlims=expressionLims_NB)
# NanobodyPaperPlotting.DAmFRETRowFromData(NBTestData, DAmFRETAxs, addTitles=True, ylims=AmFRETLims, firstColLabelsOnly=True)


for i in [0,1,2,3]:
    # remove default AmFRET==0 line
    for child in DAmFRETAxs[i].get_children():
        if isinstance(child, LineCollection):
            child.remove()
    
    DAmFRETAxs[i].hlines(0, expressionLims[0], expressionLims[1], color=(0.6,0.6,0.6), linestyle="--")


DAmFRETFig.tight_layout()
plt.show()
# DAmFRETFig.savefig("plots/NBTest_DAmFRET.png") #NOTE: titles were changed in illustrator
DAmFRETFig

### Figure S2

In [ ]:
mEos4x3912b = "data/mEosValency/exported_plate1/A4.fcs"
mEos4NB6 = "data/mEosValency/exported_plate3/D4.fcs"

DAmFRETFig = plt.Figure((3*3,3), dpi=300)
DAmFRETAxs = DAmFRETFig.subplots(1,3, subplot_kw={'projection': "scatter_density"})

mEos4x3912bData = NanobodyPaperPlotting.readDataFromFilelist([mEos4x3912b])
mEos4NB6Data = NanobodyPaperPlotting.readDataFromFilelist([mEos4NB6])

mEos4NB6DataHighBDFP, _ = NanobodyPaperPlotting.gateBDFP(mEos4NB6Data, "6x mEosNb")
mEos4NB6DataLowBDFP, _ = NanobodyPaperPlotting.gateBDFP(mEos4NB6Data, "Control")

NanobodyPaperPlotting.plotDAmFRETDensity(mEos4x3912bData["Acceptor/SSC"], mEos4x3912bData["AmFRET"], logX=True, xlims=expressionLims, ylims=AmFRETLims, ax=DAmFRETAxs[0], title="4x mEos3 Control")
NanobodyPaperPlotting.plotDAmFRETDensity(mEos4NB6DataLowBDFP["Acceptor/SSC"], mEos4NB6DataLowBDFP["AmFRET"], logX=True, xlims=expressionLims, ylims=AmFRETLims, ax=DAmFRETAxs[1], title="4x mEos3 with 6x mEosNb (bdfp-)")
NanobodyPaperPlotting.plotDAmFRETDensity(mEos4NB6DataHighBDFP["Acceptor/SSC"], mEos4NB6DataHighBDFP["AmFRET"], logX=True, xlims=expressionLims, ylims=AmFRETLims, ax=DAmFRETAxs[2], title="4x mEos3 with 6x mEosNb (BDFP+)")

for i in [1,2]:
    DAmFRETAxs[i].set_xlabel(None)
    DAmFRETAxs[i].set_ylabel(None)

DAmFRETAxs[0].set_xlabel("mEos3 concentration (p.d.u.)")

for i in [0,1,2]:
    # remove default AmFRET==0 line
    for child in DAmFRETAxs[i].get_children():
        if isinstance(child, LineCollection):
            child.remove()
    
    DAmFRETAxs[i].hlines(0, expressionLims[0], expressionLims[1], color=(0.6,0.6,0.6), linestyle="--")

DAmFRETFig.tight_layout()
# DAmFRETFig.savefig("plots/BDFPSupplement_DAmFRET.png") #NOTE: titles were changed in illustrator
plt.show()
DAmFRETFig

In [ ]:
#use this plot for the bdfp gating explanation
fig = plt.Figure((2*3,3), dpi=300)
axs = fig.subplots(1,2)


#these values were chosen by visual inspection, and are the same as what appear in NanobodyPaperPlotting.py
lowBDFPCutoff = 1
highBDFPCutoff = 2

axs[0].scatter(mEos4x3912bData["Acceptor/SSC"], mEos4x3912bData["BDFP/SSC"], s=0.5, alpha=0.2)
axs[0].hlines([1,2], expressionLims[0], expressionLims[1], color=["blue", "red"])

axs[0].set_title("4x mEos3 Control")

axs[0].set_xscale("log")
axs[0].set_yscale("symlog")
axs[0].set_ylim(-3, 10**BDFPPositiveBDFPSSCLims_kde[1])
axs[0].set_xlim(expressionLims[0], expressionLims[1])

# axs[0].set_xlabel(defaultXLabel)
axs[0].set_xlabel("mEos3 concentration (p.d.u.)") #extended label
axs[0].set_ylabel(defaultYLabel)

axs[1].scatter(mEos4NB6Data["Acceptor/SSC"], mEos4NB6Data["BDFP/SSC"], s=0.5, alpha=0.2)
axs[1].hlines([lowBDFPCutoff, highBDFPCutoff], expressionLims[0], expressionLims[1], color=["blue", "red"])

axs[1].set_title("4x mEos3 with 6x mEosNb")
axs[1].set_xscale("log")
axs[1].set_yscale("symlog")
axs[1].set_ylim(-3, 10**BDFPPositiveBDFPSSCLims_kde[1])
axs[1].set_xlim(expressionLims[0], expressionLims[1])

fig.tight_layout()
# fig.savefig("plots/BDFPSupplement_bdfpGating.png")

fig

In [ ]:
#x3912b version - we don't like this one conceptually, but it is a valid comparison

# mEos1xControlData = NanobodyPaperPlotting.readDataFromFilelist(["data/mEosValency/exported_plate1/A1.fcs"])
# mEos2xControlData = NanobodyPaperPlotting.readDataFromFilelist(["data/mEosValency/exported_plate1/A2.fcs"])
# mEos3xControlData = NanobodyPaperPlotting.readDataFromFilelist(["data/mEosValency/exported_plate1/A3.fcs"])
# mEos4xControlData = NanobodyPaperPlotting.readDataFromFilelist(["data/mEosValency/exported_plate1/A4.fcs"])

# mEos1xControlData["mEosN"] = 1
# mEos2xControlData["mEosN"] = 2
# mEos3xControlData["mEosN"] = 3
# mEos4xControlData["mEosN"] = 4

# mEosSeriesData = pd.concat([mEos1xControlData, mEos2xControlData, mEos3xControlData, mEos4xControlData])

# # blueColors = ["", "#4A566A", "#2D5C78", "#3971A5", "#3FABFF"] #don't do shades of blue - too little contrast
# tab10 = plt.get_cmap("tab10")
# reorderedTab10 = ["", tab10(0), tab10(1), tab10(2), tab10(4)]

# fig = plt.Figure((3,3), dpi=300)
# ax = fig.add_subplot(projection = "scatter_density")

# # NanobodyPaperPlotting.plotDAmFRETClusters(combinedData["Acceptor/SSC"], combinedData["AmFRET"], combinedData["mEosN"], logX=True, colors=blueColors, xlims=expressionLims, ax=ax)
# # NanobodyPaperPlotting.plotDAmFRETClusters(combinedData["Acceptor/SSC"], combinedData["AmFRET"], combinedData["mEosN"], logX=True, colors=blueColors, ylims=(-0.2,0.6), xlims=expressionLims, ax=ax)
# NanobodyPaperPlotting.plotDAmFRETClusters(mEosSeriesData["Acceptor/SSC"], mEosSeriesData["AmFRET"], mEosSeriesData["mEosN"], logX=True, ylims=(-0.2,0.75), xlims=expressionLims, ax=ax, colors=reorderedTab10, xlab="mEos3 concentration (p.d.u.)")

# for text in ax.texts:
#     text.set_visible(False)
    
# plt.show()
# # fig

# labels = ["4x mEos3", "3x mEos3", "2x mEos3", "1x mEos3"]

# textX = 0.01
# textYMax = 0.98
# #make the label for the highest pop
# label = labels[0]
# # higherText = ax.text(textX, textYMax, label, va="top", ha="left", color=blueColors[int(label[0])], transform = ax.transAxes)
# higherText = ax.text(textX, textYMax, label, va="top", ha="left", color=reorderedTab10[int(label[0])], transform = ax.transAxes)

# #make label for non-highest pop(s)
# for i, label in enumerate(labels[1:]):
#     # higherText = ax.annotate(label, xycoords=higherText, xy=(0,-1), color=blueColors[int(label[0])], horizontalalignment="left", transform = ax.transAxes)
#     higherText = ax.annotate(label, xycoords=higherText, xy=(0,-1), color=reorderedTab10[int(label[0])], horizontalalignment="left", transform = ax.transAxes)

# fig.tight_layout()
# # fig.savefig("plots/mEosSeries.png")
# fig

In [ ]:
# 1x mEosNb bdfp- version

mEos1xControlData = NanobodyPaperPlotting.readDataFromFilelist(["data/mEosValency/exported_plate2/G1.fcs"])
mEos2xControlData = NanobodyPaperPlotting.readDataFromFilelist(["data/mEosValency/exported_plate2/G2.fcs"])
mEos3xControlData = NanobodyPaperPlotting.readDataFromFilelist(["data/mEosValency/exported_plate2/G3.fcs"])
mEos4xControlData = NanobodyPaperPlotting.readDataFromFilelist(["data/mEosValency/exported_plate2/G4.fcs"])

mEos1xControlData, _ = NanobodyPaperPlotting.gateBDFP(mEos1xControlData, "Control")
mEos2xControlData, _ = NanobodyPaperPlotting.gateBDFP(mEos2xControlData, "Control")
mEos3xControlData, _ = NanobodyPaperPlotting.gateBDFP(mEos3xControlData, "Control")
mEos4xControlData, _ = NanobodyPaperPlotting.gateBDFP(mEos4xControlData, "Control")

mEos1xControlData["mEosN"] = 1
mEos2xControlData["mEosN"] = 2
mEos3xControlData["mEosN"] = 3
mEos4xControlData["mEosN"] = 4

mEosSeriesData = pd.concat([mEos1xControlData, mEos2xControlData, mEos3xControlData, mEos4xControlData])

# blueColors = ["", "#4A566A", "#2D5C78", "#3971A5", "#3FABFF"] #don't do shades of blue - too little contrast
tab10 = plt.get_cmap("tab10")
reorderedTab10 = ["", tab10(0), tab10(1), tab10(2), tab10(4)]

fig = plt.Figure((3,3), dpi=300)
ax = fig.add_subplot(projection = "scatter_density")

# NanobodyPaperPlotting.plotDAmFRETClusters(combinedData["Acceptor/SSC"], combinedData["AmFRET"], combinedData["mEosN"], logX=True, colors=blueColors, xlims=expressionLims, ax=ax)
# NanobodyPaperPlotting.plotDAmFRETClusters(combinedData["Acceptor/SSC"], combinedData["AmFRET"], combinedData["mEosN"], logX=True, colors=blueColors, ylims=(-0.2,0.6), xlims=expressionLims, ax=ax)
NanobodyPaperPlotting.plotDAmFRETClusters(mEosSeriesData["Acceptor/SSC"], mEosSeriesData["AmFRET"], mEosSeriesData["mEosN"], logX=True, ylims=(-0.2,0.75), xlims=expressionLims, ax=ax, colors=reorderedTab10, xlab="mEos3 concentration (p.d.u.)")

# remove default AmFRET==0 line
for child in ax.get_children():
    if isinstance(child, LineCollection):
        child.remove()

ax.hlines(0, expressionLims[0], expressionLims[1], color=(0.6,0.6,0.6), linestyle="--")

for text in ax.texts:
    text.set_visible(False)
    
plt.show()
# fig

labels = ["4x mEos3", "3x mEos3", "2x mEos3", "1x mEos3"]

textX = 0.03
textYMax = 0.97
#make the label for the highest pop
label = labels[0]
# higherText = ax.text(textX, textYMax, label, va="top", ha="left", color=blueColors[int(label[0])], transform = ax.transAxes)
higherText = ax.text(textX, textYMax, label, va="top", ha="left", color=reorderedTab10[int(label[0])], transform = ax.transAxes)

#make label for non-highest pop(s)
for i, label in enumerate(labels[1:]):
    # higherText = ax.annotate(label, xycoords=higherText, xy=(0,-1), color=blueColors[int(label[0])], horizontalalignment="left", transform = ax.transAxes)
    higherText = ax.annotate(label, xycoords=higherText, xy=(0,-1), color=reorderedTab10[int(label[0])], horizontalalignment="left", transform = ax.transAxes)

fig.tight_layout()
# fig.savefig("plots/mEosSeries.png")
fig

### Figures S4 and S6

The plots for these are actually produced in Figure2.ipynb and Figure3.ipynb, respectively.
This decision was made to simplify the data handling for the larger number of files, and because the files for these plots needed to have populations annotated.